In [1]:
from nsga2.estimator import NSGAIIRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('../data/lexicase_paper/d_yacht.txt', sep=',')

# DEAP interface requires X and y to be numpy arrays, not pandas dataframes
X = df.drop('label', axis=1).values
y = df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y)

estimator = NSGAIIRegressor(**{
    'pop_size'        : 20, 
    'max_gen'         : 10,
    'max_depth'       : 10,  # 8
    'max_size'        : 100, # 75
    'objectives'      : ['error', 'size'],
    'cx_prob'         : 1/7,
    'initialization'  : 'uniform',
    'pick_criteria'   : 'MCDM', # error, MCDM
    'validation_size' : 0.33,
    'simplify'        : False,
    'verbosity'       : 1,
}).fit(X_train, y_train)

gen	evals	n_simplifications	n_new_hashes	avg train error	avg train size	avg val error	avg val size	med train error	med train size	med val error	med val size	std train error	std train size	std val error	std val size	min train error	min train size	min val error	min val size	max train error	max train size	max val error	max val size
0  	20   	0                	0           	               	              	             	            	               	              	             	            	               	              	             	            	               	              	             	            	               	              	             	            
1  	20   	0                	0           	               	              	             	            	               	              	             	            	               	              	             	            	               	              	             	            	               	              	             	            
2  	20 

In [2]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse

model      = str(estimator.best_estimator_).replace("ARG", "x_")
size       = len(estimator.best_estimator_)
complexity = size
depth      = estimator.best_estimator_.height

print(model)
print(size)
print(complexity)
print(depth)

for metric, fn, (data_X, data_y) in [
    ('train_r2',  r2_score, (X_train, y_train)),
    ('test_r2',   r2_score, (X_test,  y_test )),
    ('train_mse', mse,      (X_train, y_train)),
    ('test_mse',  mse,      (X_test,  y_test )),
]:
    score = np.nan
    try:
        score = fn(estimator.predict(data_X), data_y)
        print(f"{metric} : {score}")
    except ValueError:
        print(f"(Failed to calculate {metric}")

subtract(mul3(log1p(9356416.40821584), x_5, x_3), x_2)
7
7
3
train_r2 : -1.5991588476733338
test_r2 : -2.3678095431141295
train_mse : 130.8999408591487
test_mse : 128.30055076930336
